In [1]:
import psycopg2 as pg

import pandas.io.sql as psql

import pandas as pd

import datetime

In [2]:
with pg.connect(database='uonuma_kikan',user='postgres',host='localhost',port=5432,password=12345) as conn:
    sql = 'SELECT * FROM yaku_hbv;'
    df1 = psql.read_sql(sql,conn)

In [3]:
with pg.connect(database='uonuma_kikan',user='postgres',host='localhost',port=5432,password=12345) as conn:
    sql = 'SELECT * FROM baraclude;'
    df2 = psql.read_sql(sql,conn)

In [4]:
# kenreki
with pg.connect(database='uonuma_kikan',user='postgres',host='localhost',port=5432,password=12345) as conn:
    sql = 'SELECT * FROM kenreki;'
    df3 = psql.read_sql(sql,conn)

In [5]:
#with pg.connect(database='uonuma_kikan',user='postgres',host='localhost',port=5432,password=12345) as conn:
#    sql = 'SELECT * FROM kekka;'
#    df3 = psql.read_sql(sql,conn)

In [6]:
#with pg.connect(database='uonuma_kikan',user='postgres',host='localhost',port=5432,password=12345) as conn:
#    sql = 'SELECT * FROM yoyaku;'
#    df4 = psql.read_sql(sql,conn)

In [7]:
id1 = len(df1)
id2 = len(df2)
id3 = len(df3)

In [8]:
id1

18881

In [9]:
id2

202

In [10]:
id3

11481

In [11]:
%cd C:\import

C:\import


In [12]:
%ls

 ドライブ C のボリューム ラベルがありません。
 ボリューム シリアル番号は 02F6-EB77 です

 C:\import のディレクトリ

2016/11/15  15:54    <DIR>          .
2016/11/15  15:54    <DIR>          ..
2016/11/15  14:49         1,090,026 idwh_20160404_1737_158008_PtBirthDate.xlsx
2016/11/15  13:10    <DIR>          old
2016/11/15  14:50         4,914,451 sdwh_20150916_1759_158008_kekka.xlsx
2016/11/15  14:52         1,207,902 sdwh_20150916_1954_158008_yoyaku.xlsx
2016/11/15  14:53             9,799 sdwh_20150916_2159_158008_baraclude.xlsx
2016/11/15  14:54            33,294 sdwh_20150916_2159_158008_処方オーダー数量_HBV.xlsx
2016/11/15  15:02            39,692 sdwh_20150916_2202_158008_注射オーダー数量_HBV.xlsx
2016/11/15  15:03            32,112 sdwh_20160727_1218_158008_kenreki.xlsx
2016/06/06  21:09    <DIR>          work
               7 個のファイル           7,327,276 バイト
               4 個のディレクトリ  403,327,844,352 バイトの空き領域


In [13]:
# kenreki
rk = pd.read_excel('sdwh_20160727_1218_158008_kenreki.xlsx')

In [14]:
def HBeAng(dx):
    for x in dx.index:
        if (dx.loc[x,'検査コード']==2010701)&(dx.loc[x,'検査名称']=='Ｓ／ＣＯ'):
            dx.loc[x,'検査名称']='ＨＢｅ抗原'
        elif (dx.loc[x,'検査コード']==2011401)&(dx.loc[x,'検査名称']=='Ｓ／ＣＯ'):
            dx.loc[x,'検査名称']='ＨＢｃ抗体'
        else:
            pass

In [15]:
HBeAng(rk)

In [16]:
rk['key'] = rk['文書日付_D']+' '+rk['患者ＩＤ'].apply(lambda x:str(x))+' '+rk['検査名称'].apply(lambda x:x[:5])

In [17]:
teisei = [2010100,2010200,2011400,2012001,2010702,2010802,2011601,2011701]
teiryo = [2010101,2010201,2011401,2012000,2010701,2010801,2011600,2011700]
# 2011600 has both sides of character, qualitative and quantitative.
# On 16 Nov, 2016 we added '2011701' in teisei and '2011700' in teiryo.

In [18]:
rk1 = rk[rk['検査コード'].isin(teisei)]

In [19]:
rk2 = rk[rk['検査コード'].isin(teiryo)]

In [20]:
rk1_ = rk1[['key','検査コード','検査名称','結果']]

In [21]:
rk1_.shape

(296, 4)

In [22]:
rk2_ = rk2[['患者ＩＤ', '文書日付_D', '伝票名称', '検査コード', '検査名称', '結果','結果(数値)', 'key']]

In [23]:
rk2_.shape

(296, 8)

In [24]:
rkc = pd.merge(rk2_,rk1_,on='key',how='left')

In [25]:
rkc.shape

(296, 11)

In [26]:
rkc.sort_values('文書日付_D',inplace=True)

In [27]:
rkc['文書日付'] = pd.to_datetime(rkc['文書日付_D'])

In [28]:
rkc['文書日付'] = rkc['文書日付'].apply(lambda x:x.date())

In [29]:
rkc.reset_index(inplace=True)

In [30]:
df1_a = pd.read_excel('sdwh_20150916_2159_158008_処方オーダー数量_HBV.xlsx')

In [31]:
df1_b = pd.read_excel('sdwh_20150916_2202_158008_注射オーダー数量_HBV.xlsx')

In [32]:
df1_ = pd.concat([df1_a,df1_b])

In [33]:
df2_ = pd.read_excel('sdwh_20150916_2159_158008_baraclude.xlsx')

In [34]:
df3_ = pd.read_excel('sdwh_20150916_1759_158008_kekka.xlsx')

In [35]:
df4_ = pd.read_excel('sdwh_20150916_1954_158008_yoyaku.xlsx')

In [36]:
df4_['予約日時'] = pd.to_datetime(df4_['予約日時'])

df4_ = df4_.sort_values(['患者ＩＤ','診療科コード','予約日時'])

df4_.drop_duplicates(['患者ＩＤ','診療科コード'],inplace=True)

df4_.reset_index(inplace=True)

df4_.drop('index',axis=1,inplace=True)

In [37]:
df1_ = df1_[['文書形態名', '文書タイトル', '実施日時', '文書日付', '患者ID', '患者表記名', '薬品コード', '薬品',
       '薬品単位コード', '薬品単位名', '薬品数量', '用法の日数回数', '薬品使用量', '入外区分名', '更新部署コード','更新部署名']]

In [38]:
df1_.reset_index(inplace=True)

In [39]:
len(df1_)

534

In [40]:
df1_['更新部署コード'] = df1_['更新部署コード'].fillna('')

In [41]:
# change float to integer.
df1_['更新部署コード'] = df1_['更新部署コード'].apply(lambda x:int(x) if x != '' else x)

In [42]:
#df1_['更新部署コード'] = df1_['更新部署コード'].apply(lambda x:str(x)[:-2])

In [43]:
# the function which generate ids for postscription
def idplot(wdf,wid):
    for i in wdf.index.tolist():
        wdf.loc[i,'index'] = wid+1+i

In [44]:
# yaku_hbv is a type of postscription.
idplot(df1_,id1)

In [45]:
df1_.to_csv('sdwh_20150916_2159_158008_yaku_hbv.csv',index=False)

In [1]:
df2_

In [47]:
df2_.reset_index(inplace=True)

In [48]:
# baraclude is a type of postscription.
idplot(df2_,id2)

In [2]:
df2_

In [50]:
df2_.to_csv('sdwh_20150916_2159_158008_baraclude.csv',index=False)

In [51]:
# kenreki
idplot(rkc,id3)

In [52]:
# kenreki
rkc.to_csv('sdwh_20160727_1218_158008_kenreki_mod.csv',index=False)

In [53]:
# kekka is a type of superscription.
df3_.reset_index(inplace=True)

df3_['index'] = df3_['index']+1

# for death_flg, you should change the data type into integer from float.
df3_['生存の有無_死亡チェック'] = df3_['生存の有無_死亡チェック'].apply(lambda x:int(x) if x==x else '')

df3_.to_csv('sdwh_20150916_1759_158008_kekka.csv',index=False)

In [54]:
# yoyaku is a type of superscription.
df4_.reset_index(inplace=True)

df4_['index'] = df4_['index']+1

# remove needless space on the leftmost.
# df4_['診療科コード'] = df4_['診療科コード'].apply(lambda x:x.replace(' ',''))

df4_.to_csv('sdwh_20150916_1954_158008_yoyaku.csv',index=False)

In [55]:
with pg.connect(database='uonuma_kikan',user='postgres',host='localhost',port=5432,password=12345) as conn:
    cursor = conn.cursor()
    sql = "COPY yaku_hbv FROM 'C:\import\sdwh_20150916_2159_158008_yaku_hbv.csv' WITH ENCODING 'sjis' CSV HEADER;"
    cursor.execute(sql)

In [56]:
with pg.connect(database='uonuma_kikan',user='postgres',host='localhost',port=5432,password=12345) as conn:
    cursor = conn.cursor()
    sql = "COPY baraclude FROM 'C:\import\sdwh_20150916_2159_158008_baraclude.csv' WITH ENCODING 'sjis' CSV HEADER;"
    cursor.execute(sql)

In [57]:
# kenreki
with pg.connect(database='uonuma_kikan',user='postgres',host='localhost',port=5432,password=12345) as conn:
    cursor = conn.cursor()
    sql = "COPY kenreki FROM 'C:\import\sdwh_20160727_1218_158008_kenreki_mod.csv' WITH ENCODING 'sjis' CSV HEADER;"
    cursor.execute(sql)

In [58]:
with pg.connect(database='uonuma_kikan',user='postgres',host='localhost',port=5432,password=12345) as conn:
    cursor = conn.cursor()
    sql = '''DELETE FROM kekka;
             COPY kekka FROM 'C:\import\sdwh_20150916_1759_158008_kekka.csv' WITH ENCODING 'sjis' CSV HEADER;'''
    cursor.execute(sql)

In [59]:
with pg.connect(database='uonuma_kikan',user='postgres',host='localhost',port=5432,password=12345) as conn:
    cursor = conn.cursor()
    sql = '''DELETE FROM yoyaku;
             COPY yoyaku FROM 'C:\import\sdwh_20150916_1954_158008_yoyaku.csv' WITH ENCODING 'sjis' CSV HEADER;'''
    cursor.execute(sql)